In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import nibabel as nib
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical # type: ignore # type: ignore
from skimage.transform import resize


In [2]:
print(pd.__version__)
print(np.__version__)
print(tf.__version__)
print(torch.__version__)
print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")

2.2.3
2.0.2
2.18.0
2.6.0+cpu
GPU is NOT AVAILABLE


### Preprocessing

##### Normalizing the dataset
##### Onehotencoding on segmentation Mask

In [3]:

VOLUME_DIR = "Dataset/volume_pt1" 
SEGMENTATION_DIR = "Dataset/volume_pt1/segementation"
IMG_SIZE = (128, 128) 

def load_nifti(file_path):
    """Load a NIfTI file and return the NumPy array"""
    print(f"Loading file: {file_path}") 
    nifti_img = nib.load(file_path) 
    return nifti_img.get_fdata()

def preprocess_data(volume_path, segmentation_path):
    """Load and preprocess volume and segmentation data"""
    volume_data = load_nifti(volume_path)
    segmentation_data = load_nifti(segmentation_path).astype(int)

    volume_data = (volume_data - np.min(volume_data)) / (np.max(volume_data) - np.min(volume_data))

    volume_resized = np.array([resize(slice, IMG_SIZE, mode='constant', preserve_range=True) for slice in volume_data.transpose(2, 0, 1)])
    segmentation_resized = np.array([resize(slice, IMG_SIZE, mode='constant', preserve_range=True, order=0) for slice in segmentation_data.transpose(2, 0, 1)])

   
    segmentation_onehot = to_categorical(segmentation_resized, num_classes=3)

    return volume_resized, segmentation_onehot


X_train, Y_train = [], []


image_files = sorted([f for f in os.listdir(VOLUME_DIR) if f.endswith(".nii")])
mask_files = sorted([f for f in os.listdir(SEGMENTATION_DIR) if f.endswith(".nii")])

for img_file, mask_file in zip(image_files, mask_files):
    volume_path = os.path.join(VOLUME_DIR, img_file)
    segmentation_path = os.path.join(SEGMENTATION_DIR, mask_file)
    vol, seg = preprocess_data(volume_path, segmentation_path)
    X_train.append(vol)
    Y_train.append(seg)
# Shape: (num_slices, 128, 128)
# Shape: (num_slices, 128, 128, 3)
X_train = np.concatenate(X_train, axis=0)  
Y_train = np.concatenate(Y_train, axis=0)  

print("Training data shape:", X_train.shape)
print("Segmentation mask shape:", Y_train.shape)

Loading file: Dataset/volume_pt1\volume-0.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-0.nii
Loading file: Dataset/volume_pt1\volume-1.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-1.nii
Loading file: Dataset/volume_pt1\volume-10.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-10.nii
Loading file: Dataset/volume_pt1\volume-2.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-2.nii
Loading file: Dataset/volume_pt1\volume-3.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-3.nii
Loading file: Dataset/volume_pt1\volume-4.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-4.nii
Loading file: Dataset/volume_pt1\volume-5.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-5.nii
Loading file: Dataset/volume_pt1\volume-6.nii
Loading file: Dataset/volume_pt1/segementation\segmentation-6.nii
Loading file: Dataset/volume_pt1\volume-7.nii
Loading file: Dataset/volume_pt1/segementation\segmentat

### Model Design 

#### Activation Function-Relu
#### Maxpolling



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
from tensorflow.keras.losses import categorical_crossentropy # type: ignore
import tensorflow.keras.backend as K # type: ignore
from skimage.transform import resize
import nibabel as nib
import matplotlib.pyplot as plt

# ---- Dice Coefficient and Loss ----
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# ---- U-Net Model ----
def unet_model(input_size=(128, 128, 1), num_classes=3):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u1 = UpSampling2D((2, 2))(c3)
    u1 = Concatenate()([u1, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = Concatenate()([u2, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)

    # Output Layer (Softmax for Multi-Class)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c5)

    model = Model(inputs, outputs)
    return model


### Model Training Unet Model

In [ ]:
from tensorflow.keras.optimizers import Adam # type: ignore # type: ignore
# 0.001 (default) → Good for general training
LEARNING_RATE = 0.001 

# Define and compile model
model = unet_model()
optimizer = Adam(learning_rate=LEARNING_RATE)  # Set learning rate
model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy', dice_coefficient])

# Train model
EPOCHS = 20
BATCH_SIZE = 16
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2)

In [ ]:
model.save("unet_liver_segmentation.h5")

In [4]:
import os
import numpy as np
import nibabel as nib
from tensorflow.keras.utils import to_categorical # type: ignore
# Function to load NIfTI files
def load_nifti(file_path):
    """Load a NIfTI file and return the NumPy array"""
    print(f"Loading: {file_path}")  # Debugging: Print file being loaded
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    nifti_img = nib.load(file_path)
    return nifti_img.get_fdata()

# Function to preprocess test data
def preprocess_data(test_volume_path, test_segmentation_path):
    """Load and preprocess a single test volume and its corresponding segmentation mask"""
    volume_data = load_nifti(test_volume_path)
    segmentation_data = load_nifti(test_segmentation_path).astype(int)

    # Normalize CT images (0-1)
    volume_data = (volume_data - np.min(volume_data)) / (np.max(volume_data) - np.min(volume_data))

    # Resize to (128, 128)
    from skimage.transform import resize
    IMG_SIZE = (128, 128)
    
    volume_resized = np.array([
        resize(slice, IMG_SIZE, mode='constant', preserve_range=True)
        for slice in volume_data.transpose(2, 0, 1)  # Rearrange to (slices, H, W)
    ])
    
    segmentation_resized = np.array([
        resize(slice, IMG_SIZE, mode='constant', preserve_range=True, order=0) 
        for slice in segmentation_data.transpose(2, 0, 1)
    ])

    # One-hot encode segmentation masks (background=0, liver=1, tumor=2)
    
    segmentation_onehot = to_categorical(segmentation_resized, num_classes=3)

    return volume_resized, segmentation_onehot

# ---- Load Test Data ----
test_image_path = os.path.join("test_folder") 
test_mask_path = os.path.join("test_folder", "segmentation") 

# Ensure directories exist
if not os.path.exists(test_image_path) or not os.path.exists(test_mask_path):
    raise FileNotFoundError("Test data folder or segmentation folder not found!")

# Get sorted filenames
image_files = sorted([f for f in os.listdir(test_image_path) if f.endswith(".nii")])
mask_files = sorted([f for f in os.listdir(test_mask_path) if f.endswith(".nii")])

# Check if the number of images and masks match
if len(image_files) != len(mask_files):
    raise ValueError("Mismatch between the number of test images and segmentation masks!")

X_test, Y_test = [], []

# Load and preprocess each test image and mask
for img_file, mask_file in zip(image_files, mask_files):
    volume_path = os.path.join(test_image_path, img_file)  # Correct path joining
    segmentation_path = os.path.join(test_mask_path, mask_file)  # Correct path joining

    vol, seg = preprocess_data(volume_path, segmentation_path)
    X_test.append(vol)
    Y_test.append(seg)

# Convert lists to NumPy arrays
X_test = np.array(X_test)  # Shape: (num_volumes, num_slices, 128, 128)
Y_test = np.array(Y_test)  # Shape: (num_volumes, num_slices, 128, 128, 3)

# Reshape to flatten across all slices
X_test = X_test.reshape(-1, 128, 128, 1)  # Add channel dimension
Y_test = Y_test.reshape(-1, 128, 128, 3)  # Keep segmentation masks in one-hot format

# Print shapes
print("Final Test Data Shape:", X_test.shape)  # (total_slices, 128, 128, 1)
print("Final Test Mask Shape:", Y_test.shape)  # (total_slices, 128, 128, 3)


Loading: test_folder\volume-10.nii
Loading: test_folder\segmentation\segmentation-10.nii
Final Test Data Shape: (501, 128, 128, 1)
Final Test Mask Shape: (501, 128, 128, 3)


### Loading the Model and then testing 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model # type: ignore

# ---- Load the trained model ----
model = load_model("unet_liver_segmentation.h5", compile=False)  # Load without compiling

# ---- Make predictions ----
y_pred = model.predict(X_test)  # Shape: (total_slices, 128, 128, 3)

# Convert predicted probabilities to class labels (0=background, 1=liver, 2=tumor)
y_pred_labels = np.argmax(y_pred, axis=-1)  # Shape: (total_slices, 128, 128)

# Convert actual one-hot masks to class labels
Y_test_labels = np.argmax(Y_test, axis=-1)  # Shape: (total_slices, 128, 128)

### Test Image

In [ ]:
# ---- Display some slices ----
num_slices = 3  # Number of slices to display
fig, axes = plt.subplots(num_slices, 3, figsize=(10, num_slices * 3))

for i in range(num_slices):
    slice_idx = i   # Adjust step to view different slices

    # Display CT Image
    axes[i, 0].imshow(X_test[slice_idx, :, :, 0], cmap='gray')
    axes[i, 0].set_title(f"Slice {slice_idx} - CT Image")

    # Display Actual Mask
    axes[i, 1].imshow(Y_test_labels[slice_idx], cmap='jet')
    axes[i, 1].set_title("Actual Mask")

    # Display Predicted Mask
    axes[i, 2].imshow(y_pred_labels[slice_idx], cmap='jet')
    axes[i, 2].set_title("Predicted Mask")

plt.tight_layout()
plt.show()

In [ ]:

##Dice Score is Implemented Here
import numpy as np
import tensorflow.keras.backend as K  # type: ignore
import tensorflow as tf

def iou_coefficient(y_true, y_pred, smooth=1):
    """Calculate the Intersection over Union (IoU) for binary mask."""
    y_true = K.cast(y_true, "float32")
    y_pred = K.cast(y_pred, "float32")
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def precision(y_true, y_pred):
    y_true = K.cast(y_true, "float32")
    y_pred = K.cast(y_pred, "float32")
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tp = K.sum(y_true_f * y_pred_f)
    fp = K.sum(y_pred_f) - tp
    return tp / (tp + fp + K.epsilon())

def recall(y_true, y_pred):
    y_true = K.cast(y_true, "float32")
    y_pred = K.cast(y_pred, "float32")
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tp = K.sum(y_true_f * y_pred_f)
    fn = K.sum(y_true_f) - tp
    return tp / (tp + fn + K.epsilon())

def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * (prec * rec) / (prec + rec + K.epsilon())

def dice_coefficient(y_true, y_pred, smooth=1):
    """Calculate Dice coefficient."""
    y_true = K.cast(y_true, "float32")
    y_pred = K.cast(y_pred, "float32")
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Convert predictions and ground truths to class labels
y_pred_labels = np.argmax(y_pred, axis=-1)  # Shape: (total_slices, 128, 128)
Y_test_labels = np.argmax(Y_test, axis=-1)  # Shape: (total_slices, 128, 128)

# Compute class-wise metrics
num_classes = 3  # Background, Liver, Tumor
for class_idx in range(num_classes):
    print(f"\nMetrics for Class {class_idx}:")
    y_true_class = tf.convert_to_tensor((Y_test_labels == class_idx).astype(np.float32))
    y_pred_class = tf.convert_to_tensor((y_pred_labels == class_idx).astype(np.float32))

    iou = K.eval(iou_coefficient(y_true_class, y_pred_class))
    prec = K.eval(precision(y_true_class, y_pred_class))
    rec = K.eval(recall(y_true_class, y_pred_class))
    f1 = K.eval(f1_score(y_true_class, y_pred_class))
    dice = K.eval(dice_coefficient(y_true_class, y_pred_class))

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Dice Coefficient: {dice:.4f}")

# Extra: Dice for Liver + Tumor combined
liver_tumor_true = tf.convert_to_tensor(((Y_test_labels == 1) | (Y_test_labels == 2)).astype(np.float32))
liver_tumor_pred = tf.convert_to_tensor(((y_pred_labels == 1) | (y_pred_labels == 2)).astype(np.float32))
dice_liver_tumor = K.eval(dice_coefficient(liver_tumor_true, liver_tumor_pred))
print("\nDice Coefficient for Liver + Tumor combined: {:.4f}".format(dice_liver_tumor))

#### Unet++ Model Architecture

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NestedConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(NestedConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        return x

class UNetPlusPlus2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=3):
        super(UNetPlusPlus2D, self).__init__()

        # Encoding layers
        self.enc1 = NestedConvBlock(in_channels, 64)
        self.enc2 = NestedConvBlock(64, 128)
        self.enc3 = NestedConvBlock(128, 256)
        self.enc4 = NestedConvBlock(256, 512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Decoding layers
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = NestedConvBlock(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = NestedConvBlock(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = NestedConvBlock(128, 64)

        # Final output layer (3-channel output)
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)  
        enc2 = self.enc2(self.pool(enc1))  
        enc3 = self.enc3(self.pool(enc2))  
        enc4 = self.enc4(self.pool(enc3))  

        dec3 = self.dec3(torch.cat([self.up3(enc4), enc3], dim=1))  
        dec2 = self.dec2(torch.cat([self.up2(dec3), enc2], dim=1))  
        dec1 = self.dec1(torch.cat([self.up1(dec2), enc1], dim=1))  

        return self.final_conv(dec1)  

class UNetPlusPlusSlices(nn.Module):
    def __init__(self, in_channels=1, out_channels=3):
        super(UNetPlusPlusSlices, self).__init__()
        self.unet_2d = UNetPlusPlus2D(in_channels, out_channels)

    def forward(self, x):
        if x.dim() == 3:  # If input is [1, 128, 128]
            x = x.unsqueeze(0)  # Convert to [1, 1, 128, 128]

        out = self.unet_2d(x)  # Process through UNet++
        out = out.squeeze(0)  # Remove batch dimension to get [1, 128, 128, 3]

        return out

# Example usage:
model = UNetPlusPlusSlices()

In [6]:
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)

In [7]:
print(X_train.shape, Y_train.shape)


torch.Size([5277, 128, 128]) torch.Size([5277, 128, 128, 3])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define Dice Coefficient (for evaluation)
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_flat = y_true.view(-1)
    y_pred_flat = y_pred.view(-1)
    intersection = (y_true_flat * y_pred_flat).sum()
    return (2. * intersection + smooth) / (y_true_flat.sum() + y_pred_flat.sum() + smooth)

# Hyperparameters
LEARNING_RATE = 0.01
EPOCHS = 2
BATCH_SIZE = 16  # Adjust batch size as needed
LOG_FILE = "training_log.txt"  # Log file path

# Custom Dataset class
class LiverTumorDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x_slice = self.X[idx].unsqueeze(0)  # Shape: [1, 128, 128] (Adding channel dimension)
        y_slice = self.Y[idx].argmax(dim=-1)  # Convert one-hot to class labels, Shape: [128, 128]
        return x_slice, y_slice

# Initialize dataset and dataloader
dataset = LiverTumorDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize model
model = UNetPlusPlusSlices(in_channels=1, out_channels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class segmentation
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Open log file for writing
with open(LOG_FILE, "w") as log_file:
    log_file.write("Epoch, Batch, Loss\n")  # CSV header

    # Training loop with batching
    for epoch in range(EPOCHS):
        model.train()
        epoch_loss = 0.0  

        for batch_idx, (x_batch, y_batch) in enumerate(dataloader):
            optimizer.zero_grad()

            # Move data to device
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            output = model(x_batch)  # Expected output: [BATCH_SIZE, 3, 128, 128]

            # Compute loss
            loss = criterion(output, y_batch)  # Shapes should match now

            # Backpropagation
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Write batch loss to log file
            log_file.write(f"{epoch+1}, {batch_idx+1}, {loss.item():.4f}\n")

            # Print progress every 10 slices
            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx+1}: Processed {BATCH_SIZE * (batch_idx + 1)} slices")

        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}")

print(f"Training complete. Logs saved in {LOG_FILE}")


In [ ]:
torch.save(model.state_dict(), 'Unet++.pth')

In [ ]:
model = UNetPlusPlusSlices()
model.load_state_dict(torch.load('Unet++.pth'))  # Load weights
model.eval()  # Set to evaluation mode

In [ ]:
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# Dice coefficient
def dice_score(y_true, y_pred, smooth=1e-6):
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    intersection = np.sum(y_true_flat * y_pred_flat)
    return (2. * intersection + smooth) / (np.sum(y_true_flat) + np.sum(y_pred_flat) + smooth)

# Custom test dataset for [501, 128, 128, 1] input and [501, 128, 128, 3] label
class LiverTumorTestDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.X.shape[0]  # 501 slices

    def __getitem__(self, idx):
        x = self.X[idx].permute(2, 0, 1).float()  # [1, 128, 128]
        y = self.Y[idx].argmax(dim=-1).long()     # [128, 128]
        return x, y

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Dataset and DataLoader
test_dataset = LiverTumorTestDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Containers for metrics
all_preds = []
all_trues = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)                      # [1, 1, 128, 128]
        y = y.squeeze(0).cpu().numpy()        # [128, 128]

        output = model(x.squeeze(0))          # [3, 128, 128] assuming model expects [1, 128, 128]
        pred = torch.argmax(output, dim=0).cpu().numpy()

        all_preds.append(pred)
        all_trues.append(y)

# Convert to NumPy arrays
all_preds = np.array(all_preds)  # [501, 128, 128]
all_trues = np.array(all_trues)  # [501, 128, 128]

# Evaluation metrics per class
num_classes = 3
for class_idx in range(num_classes):
    print(f"\nMetrics for Class {class_idx}:")
    y_true_class = (all_trues == class_idx).astype(np.uint8).flatten()
    y_pred_class = (all_preds == class_idx).astype(np.uint8).flatten()

    iou = jaccard_score(y_true_class, y_pred_class, zero_division=0)
    prec = precision_score(y_true_class, y_pred_class, zero_division=0)
    rec = recall_score(y_true_class, y_pred_class, zero_division=0)
    f1 = f1_score(y_true_class, y_pred_class, zero_division=0)
    dice = dice_score(y_true_class, y_pred_class)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Dice Coefficient: {dice:.4f}")

# Combined liver + tumor (class 1 + 2)
combined_true = ((all_trues == 1) | (all_trues == 2)).astype(np.uint8).flatten()
combined_pred = ((all_preds == 1) | (all_preds == 2)).astype(np.uint8).flatten()
combined_dice = dice_score(combined_true, combined_pred)
print("\nDice Coefficient for Liver + Tumor combined: {:.4f}".format(combined_dice))



In [ ]:
import matplotlib.pyplot as plt

def display_predictions(X_test, y_true_labels, y_pred_labels, num_samples=5, start_idx=0):
    """
    Visualizes predictions for given number of slices.
    
    Parameters:
    - X_test: input test images [N, 128, 128, 1]
    - y_true_labels: ground truth class labels [N, 128, 128]
    - y_pred_labels: predicted class labels [N, 128, 128]
    - num_samples: how many slices to display
    - start_idx: from which slice index to start
    """
    for i in range(start_idx, start_idx + num_samples):
        plt.figure(figsize=(12, 4))

        # Input slice
        plt.subplot(1, 3, 1)
        plt.imshow(X_test[i, :, :, 0], cmap='gray')
        plt.title(f"Input Slice {i}")
        plt.axis('off')

        # Ground truth mask
        plt.subplot(1, 3, 2)
        plt.imshow(y_true_labels[i], cmap='jet', vmin=0, vmax=2)
        plt.title("Ground Truth")
        plt.axis('off')

        # Predicted mask
        plt.subplot(1, 3, 3)
        plt.imshow(y_pred_labels[i], cmap='jet', vmin=0, vmax=2)
        plt.title("Predicted Mask")
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Example: Display first 5 slices
display_predictions(X_test, y_true_labels, y_pred_labels, num_samples=5, start_idx=0)

### Transformer Based Model Architecture
Patch Embedding: Flatten image into patches + linear projection.

Transformer Encoder: Several layers of Multi-head Self Attention + Feed-Forward.

Decoder: Upsample tokens to the original image resolution.

Final Conv Layer: To get output with 3 channels.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=1, embed_dim=256, patch_size=16):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
    
    def forward(self, x):
        x = self.proj(x)  # [B, embed_dim, H/patch, W/patch]
        x = x.flatten(2).transpose(1, 2)  # [B, N_patches, embed_dim]
        return x

class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim=256, num_heads=8, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.GELU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.ff(self.norm2(x))
        return x

class TransformerDecoder(nn.Module):
    def __init__(self, embed_dim=256, patch_size=16, out_channels=3):
        super().__init__()
        self.patch_size = patch_size
        self.output_proj = nn.Linear(embed_dim, patch_size * patch_size * out_channels)

    def forward(self, x, h, w):
        x = self.output_proj(x)  # [B, N, P*P*out_channels]
        x = x.view(x.size(0), h, w, self.patch_size, self.patch_size, -1)
        x = x.permute(0, 5, 1, 3, 2, 4).contiguous()  # [B, out_channels, H, P, W, P]
        x = x.view(x.size(0), -1, h * self.patch_size, w * self.patch_size)  # [B, C, H*P, W*P]
        return x

class TransformerSegmentationModel(nn.Module):
    def __init__(self, img_size=128, patch_size=16, in_channels=1, out_channels=3, embed_dim=256, depth=4, num_heads=8):
        super().__init__()
        self.patch_embed = PatchEmbedding(in_channels, embed_dim, patch_size)
        self.encoder = nn.Sequential(
            *[TransformerEncoderBlock(embed_dim, num_heads) for _ in range(depth)]
        )
        self.decoder = TransformerDecoder(embed_dim, patch_size, out_channels)
        self.h = img_size // patch_size
        self.w = img_size // patch_size

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)  # [1, 1, H, W]
        x = self.patch_embed(x)
        x = self.encoder(x)
        out = self.decoder(x, self.h, self.w)
        return out  # [B, 3, 128, 128]


class TransformerSegmentationSlices(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.model = TransformerSegmentationModel(**kwargs)

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)  # [1, 1, 128, 128]
        return self.model(x)


##### This is a NNUnet Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.InstanceNorm2d(out_ch),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.InstanceNorm2d(out_ch),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=2, padding=1),  # strided conv
            nn.InstanceNorm2d(out_ch),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Up, self).__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        self.conv = ConvBlock(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # Ensure size match
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class NnUNetLike2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=3):
        super(NnUNetLike2D, self).__init__()

        self.inc = ConvBlock(in_channels, 32)
        self.down1 = Down(32, 64)
        self.conv1 = ConvBlock(64, 64)

        self.down2 = Down(64, 128)
        self.conv2 = ConvBlock(128, 128)

        self.down3 = Down(128, 256)
        self.conv3 = ConvBlock(256, 256)

        self.up2 = Up(256, 128)
        self.up1 = Up(128, 64)
        self.up0 = Up(64, 32)

        self.out_conv = nn.Conv2d(32, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.conv1(self.down1(x1))
        x3 = self.conv2(self.down2(x2))
        x4 = self.conv3(self.down3(x3))

        x = self.up2(x4, x3)
        x = self.up1(x, x2)
        x = self.up0(x, x1)
        out = self.out_conv(x)
        return out

class NnUNetSlices(nn.Module):
    def __init__(self, in_channels=1, out_channels=3):
        super().__init__()
        self.model = NnUNetLike2D(in_channels, out_channels)

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)  # [1, 1, H, W]
        return self.model(x)

 Meta-Ensemble / Stacking
Use both model outputs as input features to train another model (like a shallow CNN or MLP).

This meta-model learns how to best combine their predictions.

Requires training a third model → better generalization but more complex.